# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [48]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [50]:
df["Review"][5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\\nVieux lot ou contrefaçon !?"

## LLMChain

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [15]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Linens.'

## SimpleSequentialChain

In [21]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

#prompt_template 1
prompt_1 = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

#Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt_1)

In [31]:
# Second Prompt
prompt_2 = ChatPromptTemplate.from_template(
    "Write a 20 words description of the following \
    company: {company_name}"
)

#Chain 2
chain_two = LLMChain(llm=llm, prompt=prompt_2)

In [32]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [33]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Bedding Co.
Royal Bedding Co. is a high-end bedding company that provides luxurious and comfortable products to customers seeking quality sleep solutions.

> Finished chain.


'Royal Bedding Co. is a high-end bedding company that provides luxurious and comfortable products to customers seeking quality sleep solutions.'

## SequentialChain

In [34]:
from langchain.chains import SequentialChain

In [53]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english: \
    \n\n {Review}."
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [54]:
second_prompt=ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input=English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="Summary")

In [55]:
# prompt template 3: identify original language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Language"
                      )


In [56]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {Summary}\n\nLanguage: {Language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Followup_message"
                     )


In [57]:
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables = ["English_Review", "Summary", "Language", "Followup_message"],
    verbose=True
)

In [60]:
review = df.Review[0]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.',
 'English_Review': "I purchased a king size set and my only complaint is that I wish the seller offered the set with four pillowcases. I ended up ordering an additional two pack of pillowcases to have a total of four, but the color didn't completely match. However, the customer service was fantastic and they sent me two more pillowcases that matched perfectly. Overall, I'm satisfied with the quality and coolness of the sheets considering the price.",
 'Summary': 'The reviewer bought a kin

## Router Chain

In [96]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

english_template = """You are an excellent High School English teacher.
You enjoy helping your students learn the ins and outs of English grammar.
You love to identify part of speech and grammatical rules, explaining in simple 
language what the relavant terms mean and giving at least 3 examples of proper use.
When you provide the examples to identify the specific words from the example that 
align to area of grammar that you are referring to.  You also pride yourself in your accuracy.
If you don't know the answer to something you would proudly say so.  You would never risk 
giving and incorrect or misleading answer.

Here is a question:
{input}"""

In [97]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "english grammar", 
        "description": "Good for answering english grammar questions", 
        "prompt_template": english_template
    }
]

In [98]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [99]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [100]:
destination_chains = {}

In [101]:
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [102]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [103]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [104]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [105]:
chain = MultiPromptChain(
            router_chain=router_chain,
            destination_chains=destination_chains,
            default_chain=default_chain,
            verbose=True
)

In [120]:
input = """what are some exapmles of list comprension in Python?
"""
output = chain.run(input)



> Entering new MultiPromptChain chain...
computer science: {'input': 'what are some examples of list comprehension in Python?'}
> Finished chain.


In [121]:
print(output)

As an AI language model, I can provide you with some examples of list comprehension in Python:

1. Creating a list of even numbers from 0 to 10:

even_numbers = [x for x in range(11) if x % 2 == 0]

2. Creating a list of squares of numbers from 1 to 10:

squares = [x**2 for x in range(1, 11)]

3. Creating a list of uppercase letters from a string:

string = "Hello World"
uppercase_letters = [x for x in string if x.isupper()]

4. Creating a list of tuples from two lists:

list1 = [1, 2, 3]
list2 = ['a', 'b', 'c']
tuples = [(x, y) for x in list1 for y in list2]

These are just a few examples of list comprehension in Python. List comprehension is a powerful tool that allows you to create lists in a concise and readable way.


In [109]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions
english grammar: Good for answering english grammar questions
